In [1]:
!pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 33.1 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.1
    Uninstalling transformers-4.38.1:
      Successfully uninstalled transformers-4.38.1


In [54]:
!pip install torch torchdata datasets evaluate rouge_score loralib peft awscli llmx sacremoses

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 10.9 MB/s eta 0:00:00


In [3]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' creden

In [5]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

model = MBartForConditionalGeneration.from_pretrained("yonyou-sg/mbart-large-50-one-to-many-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained("yonyou-sg/mbart-large-50-one-to-many-mmt")

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/256 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/10.9k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/992 [00:00<?, ?B/s]

In [6]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np

In [7]:
huggingface_dataset_name = "yonyou-sg/datacenter-initial-zh-en"
dataset = load_dataset(huggingface_dataset_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [8]:
dataset

DatasetDict({
    train: Dataset({
        features: ['资源编码', '简体中文(源)', '参考语言(英文)'],
        num_rows: 64000
    })
    validation: Dataset({
        features: ['资源编码', '简体中文(源)', '参考语言(英文)'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['资源编码', '简体中文(源)', '参考语言(英文)'],
        num_rows: 8000
    })
})

In [29]:
tokenizer.tokenize(df_train['简体中文(源)'][0])

['▁(', '已', '选', '%', '1', '$', 'd', '项', ')']

In [40]:
smpl = pd.DataFrame()

In [41]:
import re
def word_tokenize(text):
    """
    Split a text into words, numbers, and punctuation marks
    (for languages where words are separated by spaces)
    """
    return re.findall('(\w+|[^\w\s])', str(text))

In [60]:
# 应用预处理和分析
from tqdm.auto import tqdm, trange
src_lang_index = '简体中文(源)'
target_lang_index = '参考语言(英文)'
smpl = pd.DataFrame(dataset['train'])
smpl['src_lang_tokens'] = smpl[src_lang_index].apply(lambda x: tokenizer.tokenize(str(x))) # [tokenizer.tokenize(str(text)) for text in df_train['简体中文(源)']] #
smpl['target_lang_tokens'] = smpl[target_lang_index].apply(lambda x: tokenizer.tokenize(str(x)))
smpl['src_lang_words'] = smpl[src_lang_index].apply(word_tokenize)
smpl['target_lang_words'] = smpl[target_lang_index].apply(word_tokenize)

# 统计
try:
    stats = smpl[['src_lang_tokens', 'target_lang_tokens', 'src_lang_words', 'target_lang_words']].applymap(len).describe()
    print("原始语言token转化比：",stats['src_lang_tokens']['mean'] / stats['src_lang_words']['mean'])
    print("目标语言token转化比：",stats['target_lang_tokens']['mean'] / stats['target_lang_words']['mean'])
except TypeError as e:
    print(f"Caught an error: {e}")

原始语言token转化比： 2.0835989518450106
目标语言token转化比： 1.198068656289031


In [61]:
from sacremoses import MosesPunctNormalizer
import sys
import unicodedata

mpn = MosesPunctNormalizer(lang="en")
mpn.substitutions = [
    (re.compile(r), sub) for r, sub in mpn.substitutions
]

# 查找包含未知符号的文本
texts_with_unk = [
    text for text in tqdm(smpl[src_lang_index].to_list())
    if tokenizer.unk_token_id in tokenizer(str(text)).input_ids
]
print("未知符号数量：", len(texts_with_unk))

def get_non_printing_char_replacer(replace_by: str = " "):
    non_printable_map = {
        ord(c): replace_by
        for c in (chr(i) for i in range(sys.maxunicode + 1))
        # same as \p{C} in perl
        # see https://www.unicode.org/reports/tr44/#General_Category_Values
        if unicodedata.category(c) in {"C", "Cc", "Cf", "Cs", "Co", "Cn"}
    }

    def replace_non_printing_char(line) -> str:
        return line.translate(non_printable_map)

    return replace_non_printing_char

replace_nonprint = get_non_printing_char_replacer(" ")

def preproc(text):
    clean = mpn.normalize(text)
    clean = replace_nonprint(clean)
    # replace 𝓕𝔯𝔞𝔫𝔠𝔢𝔰𝔠𝔞 by Francesca
    clean = unicodedata.normalize("NFKC", clean)
    return clean
# 应用进一步的文本清洗
texts_with_unk_normed = [text for text in tqdm(texts_with_unk) if tokenizer.unk_token_id in tokenizer(preproc(str(text))).input_ids]
print("处理掉非标准标点符号后，未知符号数量：", len(texts_with_unk_normed))

# 打印统计结果
print(stats)

  0%|          | 0/64000 [00:00<?, ?it/s]

未知符号数量： 3


  0%|          | 0/3 [00:00<?, ?it/s]

处理掉非标准标点符号后，未知符号数量： 3
       src_lang_tokens  target_lang_tokens  src_lang_words  target_lang_words
count     64000.000000        64000.000000    64000.000000       64000.000000
mean         15.157531           16.082031        7.274687          13.423297
std          19.286076           20.449812       12.811202          18.155127
min           1.000000            1.000000        0.000000           0.000000
25%           6.000000            6.000000        1.000000           5.000000
50%          11.000000           12.000000        5.000000          10.000000
75%          19.000000           20.000000        9.000000          17.000000
max         865.000000          721.000000      599.000000         771.000000


In [63]:
from collections import Counter
import sentencepiece as spm
from sentencepiece import sentencepiece_model_pb2 as sp_pb2_model
from transformers import NllbTokenizer, AutoModelForSeq2SeqLM

model_name = "yonyou-sg/mbart-large-50-one-to-many-mmt"
all_texts = smpl[src_lang_index].dropna().tolist() + smpl[target_lang_index].dropna().tolist()
all_text_normalized = [preproc(t) for t in tqdm(all_texts)]
chars_cnt = Counter(c for t in all_text_normalized for c in t)
required_chars = ''.join([
    k for k, v in chars_cnt.most_common()
    if v >= 4 and k not in ' '
])
# 我们将文本转储到一个纯文本文件中，并在此文件上训练一个新的句子分词器模型，以便将其标记添加到现有的MBart分词器中。Sentencepiece是训练分词器的流行算法之一。
all_texts_file = 'all_texts_plain.txt'
SPM_PREFIX = 'spm_new_text_16k'
with open(all_texts_file, 'w') as f:
    for i, text in enumerate(all_texts):
        print(text, file=f)

spm.SentencePieceTrainer.train(
    input=all_texts_file,
    model_prefix=SPM_PREFIX,
    vocab_size=2**14,  # 16K
    character_coverage = 1,
    num_threads=16,
    train_extremely_large_corpus=False,
    add_dummy_prefix=False,
    max_sentencepiece_length=128,
    max_sentence_length=4192*4,
    pad_id=0,
    eos_id=1,
    unk_id=2,
    bos_id=-1,
    required_chars=required_chars,
)
# 读取MBart分词器和新训练的分词器模型
tokenizer = MBart50TokenizerFast.from_pretrained(model_name)
sp_trained = spm.SentencePieceProcessor(model_file=f'{SPM_PREFIX}.model')
added_spm = sp_pb2_model.ModelProto()
added_spm.ParseFromString(sp_trained.serialized_model_proto())
old_spm = sp_pb2_model.ModelProto()
old_spm.ParseFromString(tokenizer.sp_model.serialized_model_proto())

# 将缺失的tokens加入MBart分词器模型
mbart_tokens_set = {p.piece for p in old_spm.pieces}
prev_min_score = old_spm.pieces[-1].score
for p in added_spm.pieces:
    piece = p.piece
    if piece not in mbart_tokens_set:
        new_p = sp_pb2_model.ModelProto().SentencePiece()
        new_p.piece = piece
        # for all new tokens, I'll set a lower score (priority)
        new_p.score = p.score + prev_min_score
        old_spm.pieces.append(new_p)

# 保存结果
NEW_SPM_NAME = 'spm_mbart_new.model'
with open(NEW_SPM_NAME, 'wb') as f:
    f.write(old_spm.SerializeToString())

# 加载tokenizers
tokenizer_old = tokenizer = MBart50TokenizerFast.from_pretrained(model_name)
tokenizer = MBart50TokenizerFast.from_pretrained(model_name, vocab_file=NEW_SPM_NAME)
print('原始版本分词器词汇量：',len(tokenizer_old),'新版本分词器词汇量：',len(tokenizer))
added_vocab = set(tokenizer.get_vocab()).difference(set(tokenizer_old.get_vocab()))
print('新增的词汇量：',len(added_vocab))

# 加载并调整embedding层大小
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model.resize_token_embeddings(len(tokenizer))

# 初始化新的embedding层
for t in tqdm(added_vocab):
    tt = tokenizer_old(t, add_special_tokens=False).input_ids
    if len(tt) == 0:
        tt = [tokenizer_old.unk_token_id]
    idx = tokenizer.convert_tokens_to_ids(t)
    model.model.shared.weight.data[idx] = model.model.shared.weight.data[tt].mean(0)

  0%|          | 0/127997 [00:00<?, ?it/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'MBart50Tokenizer'. 
The class this function is called from is 'NllbTokenizer'.
the `lang_code_to_id` attribute is deprecated. The logic is natively handled in the `tokenizer.adder_tokens_decoder` this attribute will be removed in `transformers` v4.38
the `fairseq_tokens_to_ids` attribute is deprecated. The logic is natively handled in the `tokenizer.adder_tokens_decoder` this attribute will be removed in `transformers` v4.38
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


原始版本分词器词汇量： 250054 新版本分词器词汇量： 250054
新增的词汇量： 0


0it [00:00, ?it/s]

In [9]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(model))

trainable model parameters: 610879488
all model parameters: 610879488
percentage of trainable model parameters: 100.00%


In [10]:
args = Seq2SeqTrainingArguments(output_dir="./mbart_yonyou/",
                        do_train=True,
                        remove_unused_columns=False,
                        do_eval=True,
                        evaluation_strategy="epoch",
                        per_device_train_batch_size=16,
                        per_device_eval_batch_size=16,
                        learning_rate=5e-5,
                        num_train_epochs=1,
                        logging_dir="/logs")

In [11]:
dataset

DatasetDict({
    train: Dataset({
        features: ['资源编码', '简体中文(源)', '参考语言(英文)'],
        num_rows: 64000
    })
    validation: Dataset({
        features: ['资源编码', '简体中文(源)', '参考语言(英文)'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['资源编码', '简体中文(源)', '参考语言(英文)'],
        num_rows: 8000
    })
})

In [12]:
train_dataset, eval_dataset = dataset["train"], dataset["validation"]

In [13]:
def data_collator(features):
    labels_key = '简体中文(源)'
    inputs_key = '参考语言(英文)'

    # 从列表中提取数据
    labels = [str(f[labels_key]) for f in features]
    inputs = [str(f[inputs_key]) for f in features]

    # 检查是否有足够的样本
    if len(labels) == 0 or len(inputs) == 0:
        raise ValueError("训练数据为空。")

    # 使用 tokenizer.prepare_seq2seq_batch 处理数据
    batch = tokenizer.prepare_seq2seq_batch(
        src_texts=inputs,
        src_lang="en_XX",
        tgt_lang="zh_CN",
        tgt_texts=labels,
        max_length=32,
        max_target_length=32
    )

    # 将数据转换为tensor
    for k in batch:
        batch[k] = torch.tensor(batch[k])

    return batch

In [14]:
trainer = Seq2SeqTrainer(model=model,
                args=args,
                data_collator=data_collator,
                # train_dataset=train_dataset.select(range(25600,64000)),
                train_dataset=train_dataset,
                eval_dataset=eval_dataset)

In [15]:
import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")

In [16]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.310700,0.303191


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#sav

TrainOutput(global_step=4000, training_loss=0.4265455360412598, metrics={'train_runtime': 1550.1643, 'train_samples_per_second': 41.286, 'train_steps_per_second': 2.58, 'total_flos': 4232268991168512.0, 'train_loss': 0.4265455360412598, 'epoch': 1.0})

这里会显示我们训练的模型运行结果

In [17]:
model =  MBartForConditionalGeneration.from_pretrained('./mbart_yonyou/checkpoint-4000')

In [18]:
text = "The effective unit price precision [%s] in Org currency is greater than the currency unit price precision [%s]"

In [19]:
encoded_batch = tokenizer(text, return_tensors="pt", padding=True)
generated_tokens = model.generate(
    **encoded_batch,
    forced_bos_token_id=tokenizer.lang_code_to_id["zh_CN"],
)
translated_batch = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

In [20]:
translated_batch

['组织币有效单价精度[%s]大于币种单价精度[%s]']

这里会显示我们原来的模型运行结果

In [22]:
original_model = MBartForConditionalGeneration.from_pretrained("yonyou-sg/mbart-large-50-one-to-many-mmt")

In [23]:
encoded_batch = tokenizer(text, return_tensors="pt", padding=True)
generated_tokens = original_model.generate(
    **encoded_batch,
    forced_bos_token_id=tokenizer.lang_code_to_id["zh_CN"],
)
translated_batch = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

In [24]:
translated_batch

['Org货币的有效单元价格精确度[%s]大于货币单元价格精确度[%s]']

可以看到效果好了很多，最后，我们把模型保存到我们的模型仓库中，名字自己定义，tokenizer也要保存

In [ ]:
model.push_to_hub("yonyou-sg/mbart-large-50-one-to-many-mmt-finetuned")

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/princepride4/mbart-large-50-one-to-many-mmt-finetuned/commit/5850e0353d37c463ca36dd526a5e64099515d78e', commit_message='Upload MBartForConditionalGeneration', commit_description='', oid='5850e0353d37c463ca36dd526a5e64099515d78e', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
tokenizer.push_to_hub("yonyou-sg/mbart-large-50-one-to-many-mmt-finetuned")

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/princepride4/mbart-large-50-one-to-many-mmt-finetuned/commit/f0a8a31450128e12986d069a1228a77135088be1', commit_message='Upload tokenizer', commit_description='', oid='f0a8a31450128e12986d069a1228a77135088be1', pr_url=None, pr_revision=None, pr_num=None)